<a href="https://colab.research.google.com/github/tongnamuu/Tensorflow_Tutorials/blob/master/Tensorflow_tutorials_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

%tensorflow_version 2.x
import numpy as np
import tensorflow as tf

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

np.set_printoptions(precision=3, suppress=True)

In [38]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [0]:
LABEL_COLUMN='survived'
LABELS=[0,1]

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [41]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'female' b'female']
age                 : [34. 28. 27. 28. 28.]
n_siblings_spouses  : [0 0 0 0 3]
parch               : [0 0 2 0 1]
fare                : [ 8.05   7.75  11.133  7.229 25.467]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'y' b'n' b'y' b'n']


In [42]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [65. 10. 34. 23. 44.]
n_siblings_spouses  : [0 3 1 0 2]
parch               : [1 2 0 0 0]
fare                : [61.979 27.9   26.     7.896 90.   ]
class               : [b'First' b'Third' b'Second' b'Third' b'First']
deck                : [b'B' b'unknown' b'unknown' b'unknown' b'C']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'n' b'n' b'n' b'y' b'n']


In [43]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [ 9. 17. 51. 48. 36.]
n_siblings_spouses  : [4 4 0 0 1]
class               : [b'Third' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'n' b'n' b'y' b'y' b'n']


In [44]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 27. 23. 39. 51.]
n_siblings_spouses  : [0. 0. 0. 1. 1.]
parch               : [0. 0. 0. 5. 0.]
fare                : [ 7.225 30.5   13.    31.275 77.958]


In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [47]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[32.     0.     0.     7.925]
 [ 4.     1.     1.    11.133]
 [30.     0.     0.    56.929]
 [ 6.     0.     1.    12.475]
 [28.     0.     0.     7.229]]

[0 1 1 1 1]


In [48]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
age                 : [20. 58. 29. 11. 35.]
n_siblings_spouses  : [0 0 1 1 0]
parch               : [0 1 1 2 0]
fare                : [  7.854 153.462  10.462 120.      7.05 ]
class               : [b'Third' b'First' b'Third' b'First' b'Third']
deck                : [b'unknown' b'C' b'G' b'B' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'n' b'n' b'y']


In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [51]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

show_batch(packed_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'female']
class               : [b'Third' b'Third' b'First' b'First' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'C' b'C']
embark_town         : [b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'y' b'n']
numeric             : [[ 24.      0.      3.     19.258]
 [ 20.      0.      0.      9.5  ]
 [ 42.      0.      0.    227.525]
 [ 28.      0.      0.     30.5  ]
 [ 17.      1.      0.    108.9  ]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [53]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [55]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f87d6c59400>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [56]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[45.   ,  1.   ,  0.   , 83.475],
       [18.   ,  0.   ,  0.   , 73.5  ],
       [34.   ,  1.   ,  1.   , 32.5  ],
       [28.   ,  0.   ,  0.   , 79.2  ],
       [22.   ,  0.   ,  0.   ,  7.775]], dtype=float32)>

In [57]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 1.228,  0.395, -0.479,  0.899],
       [-0.93 , -0.474, -0.479,  0.716],
       [ 0.349,  0.395,  0.782, -0.035],
       [-0.13 , -0.474, -0.479,  0.821],
       [-0.61 , -0.474, -0.479, -0.487]], dtype=float32)

In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [60]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [61]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [62]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     1.     0.     0.     0.     0.     1.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     1.228  0.395
 -0.479  0.899  1.     0.   ]


In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [64]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 1s 12ms/step - loss: 0.4946 - accuracy: 0.7496
Epoch 2/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4230 - accuracy: 0.8166
Epoch 3/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4005 - accuracy: 0.8278
Epoch 4/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3900 - accuracy: 0.8246
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3754 - accuracy: 0.8405
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3748 - accuracy: 0.8453
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3676 - accuracy: 0.8405
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3616 - accuracy: 0.8421
Epoch 9/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3544 - accuracy: 0.8501
Epoch 10/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3479 - accuracy: 0.851

In [65]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     53/Unknown - 1s 11ms/step - loss: 0.4532 - accuracy: 0.8409

Test Loss 0.45323561490425524, Test Accuracy 0.8409090638160706


In [66]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0])," | Actual outcome: ",("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: -109.38%  | Actual outcome:  SURVIVED
Predicted survival: 472.54%  | Actual outcome:  DIED
Predicted survival: -229.50%  | Actual outcome:  DIED
Predicted survival: 51.34%  | Actual outcome:  DIED
Predicted survival: 747.89%  | Actual outcome:  DIED
